In [1]:
import torch
import torchvision.transforms as transforms 
import torch.nn.init
from torchsummary import summary as summary_
import json
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data
import torchvision.datasets as datasets

In [2]:
#Train data 받아오기

path_dir = 'LJH/data/product_data/train/Item-Parse_f' #Parse 폴더에서 파일명 불러오기
file_list_train = os.listdir(path_dir)
file_list_train

['1029391_B.json',
 '1112498_B.json',
 '1028177_B.json',
 '1020155_F.json',
 '1211013_F.json',
 '1202318_B.json',
 '1106198_B.json',
 '1013308_F.json',
 '1203208_B.json',
 '1117018_B.json',
 '1102028_F.json',
 '1126265_F.json',
 '1116527_B.json',
 '1015504_F.json',
 '1210536_B.json',
 '1106310_B.json',
 '1110001_F.json',
 '1112457_F.json',
 '1113098_B.json',
 '1117398_F.json',
 '1118168_B.json',
 '1207459_B.json',
 '1202394_B.json',
 '1207226_B.json',
 '1210501_B.json',
 '1110257_F.json',
 '1105665_F.json',
 '1111228_B.json',
 '1201441_F.json',
 '1016098_B.json',
 '1209278_F.json',
 '1126164_B.json',
 '1021215_F.json',
 '1124325_B.json',
 '1127276_B.json',
 '1023287_F.json',
 '1111019_B.json',
 '1201414_F.json',
 '1109545_B.json',
 '1022058_B.json',
 '1027403_B.json',
 '1211241_B.json',
 '1030308_B.json',
 '1028308_F.json',
 '1120072_B.json',
 '1210797_F.json',
 '1013179_B.json',
 '1015155_B.json',
 '1027136_B.json',
 '1029395_B.json',
 '1117139_F.json',
 '1126300_B.json',
 '1118078_F.

In [3]:
#TRAIN_SIZE = 33170
TRAIN_SIZE = 10000

#label = [0 for i in range(len(file_list_train))] #list형
label = [0 for i in range(10000)]
path_dir_img = 'LJH/data/product_data/train/Item-Image'

def make_file_list():
    #train_img_list = [0 for i in range(len(file_list_train))]
    train_img_list = [0 for i in range(10000)]
    cnt = 0
    for i in range(10000):
    
        with open(path_dir +'/'+ file_list_train[cnt], "r") as st_json:
            load_data = json.load(st_json)

        category = load_data.get('category_id') #class 찾아오기
        #print(category)
        str = file_list_train[cnt].split('.')[0]
        train_img_list[cnt] = path_dir_img +'/'+ str+".jpg" #이미지파일 파일명 생성
        label[cnt] = category

        cnt+=1

    return train_img_list

In [4]:
"""train_meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x, _ in train_dataset]
train_stdRGB = [np.std(x.numpy(), axis=(1,2)) for x, _ in train_dataset]

train_meanR = np.mean([m[0] for m in train_meanRGB])
train_meanG = np.mean([m[1] for m in train_meanRGB])
train_meanB = np.mean([m[2] for m in train_meanRGB])
train_stdR = np.mean([s[0] for s in train_stdRGB])
train_stdG = np.mean([s[1] for s in train_stdRGB])
train_stdB = np.mean([s[2] for s in train_stdRGB])



print(train_meanR, train_meanG, train_meanB)"""

'train_meanRGB = [np.mean(x.numpy(), axis=(1,2)) for x, _ in train_dataset]\ntrain_stdRGB = [np.std(x.numpy(), axis=(1,2)) for x, _ in train_dataset]\n\ntrain_meanR = np.mean([m[0] for m in train_meanRGB])\ntrain_meanG = np.mean([m[1] for m in train_meanRGB])\ntrain_meanB = np.mean([m[2] for m in train_meanRGB])\ntrain_stdR = np.mean([s[0] for s in train_stdRGB])\ntrain_stdG = np.mean([s[1] for s in train_stdRGB])\ntrain_stdB = np.mean([s[2] for s in train_stdRGB])\n\n\n\nprint(train_meanR, train_meanG, train_meanB)'

In [5]:
class ImageTransform():

    def __init__(self):
        
        self.data_transform = transforms.Compose([
            transforms.CenterCrop(720),
            transforms.Resize(256),
            #transforms.Normalize([train_meanR, train_meanG, train_meanB], [train_stdR, train_stdG, train_stdB]),
            transforms.ToTensor(),
        ])
        

    def __call__(self, img):
        return self.data_transform(img)

In [6]:
class Img_Dataset(data.Dataset):

    def __init__(self, file_list, transform):
        self.file_list = file_list
        self.transform = transform

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, index):
        img_path = self.file_list[index]
        img = Image.open(img_path)
        img_transformed = self.transform(img)
        img_label = label[index]

        return img_transformed,img_label

In [7]:
train_img_list=make_file_list()

In [8]:
train_dataset = Img_Dataset(file_list=train_img_list,
                            transform=ImageTransform())

In [9]:
train_dataloader = data.DataLoader(train_dataset,
                                   batch_size=128,
                                   shuffle=True)
print('Dataloader created')

Dataloader created


In [10]:
batch_iterator = iter(train_dataloader)
images = next(batch_iterator)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [12]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()
        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BasicBlock.expansion, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(out_channels * BasicBlock.expansion),
        )

        self.shortcut = nn.Sequential()
        self.relu = nn.ReLU()

        if stride != 1 or in_channels != BasicBlock.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels * BasicBlock.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * BasicBlock.expansion)
            )

    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x


class BottleNeck(nn.Module):
    expansion = 4
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()

        self.residual_function = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels * BottleNeck.expansion, kernel_size=1, stride=1, bias=False),
            nn.BatchNorm2d(out_channels * BottleNeck.expansion),
        )

        self.shortcut = nn.Sequential()

        self.relu = nn.ReLU()

        if stride != 1 or in_channels != out_channels * BottleNeck.expansion:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels*BottleNeck.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels*BottleNeck.expansion)
            )
            
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        x = self.relu(x)
        return x

In [13]:
class ResNet(nn.Module):
    def __init__(self, block, num_block, num_classes=5, init_weights=True):
        super().__init__()

        self.in_channels=64

        self.conv1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )

        self.conv2_x = self._make_layer(block, 64, num_block[0], 1)
        self.conv3_x = self._make_layer(block, 128, num_block[1], 2)
        self.conv4_x = self._make_layer(block, 256, num_block[2], 2)
        self.conv5_x = self._make_layer(block, 512, num_block[3], 2)

        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # weights inittialization
        if init_weights:
            self._initialize_weights()

    def _make_layer(self, block, out_channels, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_channels, out_channels, stride))
            self.in_channels = out_channels * block.expansion

        return nn.Sequential(*layers)

    def forward(self,x):
        output = self.conv1(x)
        output = self.conv2_x(output)
        x = self.conv3_x(output)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

    # define weight initialization function
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)

def resnet18():
    return ResNet(BasicBlock, [2,2,2,2])

def resnet34():
    return ResNet(BasicBlock, [3, 4, 6, 3])

def resnet50():
    return ResNet(BottleNeck, [3,4,6,3])

def resnet101():
    return ResNet(BottleNeck, [3, 4, 23, 3])

def resnet152():
    return ResNet(BottleNeck, [3, 8, 36, 3])

In [14]:
from torchsummary import summary as summary_
model = resnet50().to(device)
summary_(model,(3,256,256),batch_size = 128)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [128, 64, 128, 128]           9,408
       BatchNorm2d-2        [128, 64, 128, 128]             128
              ReLU-3        [128, 64, 128, 128]               0
         MaxPool2d-4          [128, 64, 64, 64]               0
            Conv2d-5          [128, 64, 64, 64]           4,096
       BatchNorm2d-6          [128, 64, 64, 64]             128
              ReLU-7          [128, 64, 64, 64]               0
            Conv2d-8          [128, 64, 64, 64]          36,864
       BatchNorm2d-9          [128, 64, 64, 64]             128
             ReLU-10          [128, 64, 64, 64]               0
           Conv2d-11         [128, 256, 64, 64]          16,384
      BatchNorm2d-12         [128, 256, 64, 64]             512
           Conv2d-13         [128, 256, 64, 64]          16,384
      BatchNorm2d-14         [128, 256,

/home/rpc/venv_envs/junha/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [15]:
print(output.size())

torch.Size([3, 5])


In [16]:
criterion = torch.nn.CrossEntropyLoss().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [17]:
total_batch = len(train_dataloader)
for epoch in range(10): 
    avg_cost = 0 
    cnt = 0
    
    for X, Y in train_dataloader: 
        X = X.to(device) 
        Y = Y.to(device) 
        
        optimizer.zero_grad() 
        hypothesis = model(X) 
        cost = criterion(hypothesis, Y) 
        cost.backward() 
        optimizer.step() 
       
        avg_cost += cost / total_batch
        if cnt% 100 == 0:
            print(cnt)
        cnt+=1
           
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

0
[Epoch:    1] cost = 0.759105563
0
[Epoch:    2] cost = 0.597411931
0
[Epoch:    3] cost = 0.546455443
0
[Epoch:    4] cost = 0.450460017
0
[Epoch:    5] cost = 0.385723889
0
[Epoch:    6] cost = 0.343593031
0
[Epoch:    7] cost = 0.321344048
0
[Epoch:    8] cost = 0.31049484
0
[Epoch:    9] cost = 0.313588023
0
[Epoch:   10] cost = 0.279601812


In [18]:
path_dir = 'LJH/data/product_data/test/Item-Parse_f'
path_dir_img = 'LJH/data/product_data/test/Item-Image'
file_list_test = os.listdir(path_dir)
#label_test = [0 for i in range(len(file_list_test))] #list형
label_test = [0 for i in range(500)] 
def make_file_list_test():
    #test_img_list = [0 for i in range(len(file_list_test))]
    test_img_list = [0 for i in range(500)]
    cnt = 0
    #for i in file_list_test:
    for i in range(500):
    
        with open(path_dir +'/'+ file_list_test[cnt], "r") as st_json:
            load_data = json.load(st_json)

        category = load_data.get('category_id') #class 찾아오기
        #print(category)
        str = file_list_test[cnt].split('.')[0]
        test_img_list[cnt] = path_dir_img +'/'+ str+".jpg" #이미지파일 파일명 생성
        label_test[cnt] = category

        cnt+=1

    return test_img_list

In [19]:
test_img_list=make_file_list_test()

In [20]:
from torchvision.transforms import ToTensor
X_test = torch.empty(0,3,256,256)


image_w = 256 
image_h = 256
cnt_img = 0
for j in test_img_list:
    image = Image.open(test_img_list[cnt_img])
    image = image.crop((0,280,720,1000))
    image = image.resize((256, 256))
    data = ToTensor()(image).unsqueeze(0)
    X_test =torch.cat([X_test,data],dim=0)
    cnt_img += 1
    if cnt_img % 1000 == 0:
        print(cnt_img)

In [21]:
Y_test = torch.tensor(label_test)

In [22]:
with torch.no_grad(): 
    X_test = X_test.to(device)
    Y_test = Y_test.to(device)
    prediction = model(X_test) 
    print(prediction.size()) 
    correct_prediction = torch.argmax(prediction, 1) == Y_test 
    accuracy = correct_prediction.float().mean() 
    print('Accuracy: ', accuracy.item())

torch.Size([500, 5])
Accuracy:  0.9300000667572021
